In [1]:
import numpy as np
import torch
import torch.nn as nn
import os
from simple_tokenizer import SimpleTokenizer as _Tokenizer


In [2]:
device = 'cpu'
save_path = f'../onnx_models_{device}'
os.makedirs(save_path, exist_ok = True)

In [3]:
_tokenizer = _Tokenizer()

In [4]:
# 1. 아스키 코드화
string = "a smile cat"
ascii_codes = np.array([ord(char) for char in string])
ascii_codes = torch.tensor(ascii_codes)
print(ascii_codes)  # 출력: [72 101 108 108 111]

# 2. 나머지 '-1'로 채워서, [1,100] 사이즈의 tensor화
arr = torch.full((1, 100), -1).to(device = device)
arr[0, :ascii_codes.size(0)] = ascii_codes
print(arr)

tensor([ 97,  32, 115, 109, 105, 108, 101,  32,  99,  97, 116])
tensor([[ 97,  32, 115, 109, 105, 108, 101,  32,  99,  97, 116,  -1,  -1,  -1,
          -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
          -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
          -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
          -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
          -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
          -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
          -1,  -1]])


-----
#### 💛 Conversion

In [5]:
import onnx
import onnxruntime
from onnxruntime.quantization import QuantType
from onnxruntime.quantization.quantize import quantize_dynamic

In [6]:
class TokenizerModule(nn.Module):
    def __init__(self, _tokenizer, device):
        super().__init__()
        self.tokenizer = _tokenizer
        
    def forward(self, x):
        x = x[x != -1]
        llen = 77 - x.shape[0]
        result = torch.zeros(1, x.shape[0] + llen, dtype=torch.int64)
        re = self.tokenizer.encode(''.join([chr(_i) for _i in x]))
        result[0][0] = 49407
        for i in range(len(re)):
            result[0][i+1] = re[i]
        result[0][i+2] = 49406
        return result


 # onnx conversion
os.makedirs(f'{save_path}/tokenizer/', exist_ok= True)
torch.onnx.export(
    model               =   TokenizerModule(_tokenizer, device),                            # 실행될 모델
    args                =   (arr),        # 모델 입력값(tuple or 여러 입력값)
    f                   =   f'{save_path}/tokenizer/to_origin.onnx',                     # 모델 저장 경로
    export_params       =   True,                 # 모델 파일 안에 학습된 모델 가중치 저장 여부
    opset_version       =   14,                   # 모델 변환할 때 사용할 onnx 버전
    do_constant_folding =   True,         # 최적화시 상수폴딩 사용할지 여부
    input_names     =   ['input'],
    output_names    =   ["output"],
    dynamic_axes    =   {
        'input'     : {0 : 'batch_size'},    # 가변적인 길이를 가진 차원
    }
) 
# model quantization
quantize_dynamic(
    model_input     =   f'./{save_path}/tokenizer/to_origin.onnx', 
    model_output    =   f'./{save_path}/tokenizer/to_quant.onnx', 
    per_channel     =   False,
    reduce_range    =   False,
    weight_type     =   QuantType.QUInt8,
)

/tmp/ipykernel_3571847/3836549866.py:10: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  re = self.tokenizer.encode(''.join([chr(_i) for _i in x]))
  elem_type: 7
  shape {
    dim {
      dim_value: 2
    }
    dim {
      dim_param: "unk__0"
    }
  }
}
.


---
#### 💚 ONNX-Runtime Test

In [9]:
import onnxruntime as ort

# Load the ONNX model
onnx_model_path = f'{save_path}/tokenizer/to_quant.onnx'
session = ort.InferenceSession(onnx_model_path)

# Print the input names and shapes
input_names = [input.name for input in session.get_inputs()]
output_names = [output.name for output in session.get_outputs()]

print("Input names:", input_names)
print("Output names:", output_names)

Input names: ['input']
Output names: ['output']


In [10]:
# test running
ort_inputs  = {'input': arr.detach().cpu().numpy()}
ort_outputs = session.run(None, ort_inputs)
print(ort_outputs[0].shape)
print(ort_outputs[0])

(1, 77)
[[49407   320  3490  2368 49406     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]]
